In [2]:
from openpyxl import load_workbook
from datetime import datetime, timedelta
from calendar import monthrange
import os
import pandas as pd
import xlrd
import numpy as np
import re
import openpyxl 
import math


### Creation of final table column names ###

original_column_names = ['19G1_MW', '19G1_MX', '19G1_KV', '19G1_AMPS', '19G1_AMPS2', '19G1_VOLTS', '19G1_PF', '19G1_RPM', '19G1_GP', '19G1_AVR_BAL', '19G1_Trans_Temp', '19G1_TR_WDG']
# Initialize a list to store new column names
column_names = ['Date&Time']

number_of_units = 4

# Iterate over original column names 
for i in range(1,number_of_units + 1):
    for column_name in original_column_names:  
        # Change 'G1' to 'G2', 'G3', 'G4' for subsequent duplicates
        new_name = column_name.replace('G1', 'G' + str(i))
        column_names.append(new_name)

# Create DataFrame with new column names
all_sheets_df = pd.DataFrame(columns=column_names)

### Excel File Processing and Validation Functions for Creating DataFrames For The Raw Data ###

# Create a list of visible sheet names from an .xls file
def read_visible_sheets_xls(file_path):
    visible_sheet_names = []
    with xlrd.open_workbook(file_path) as workbook:
        for sheet_name in pd.ExcelFile(file_path).sheet_names:
            sheet = workbook.sheet_by_name(sheet_name)
            if not sheet.visibility:
                visible_sheet_names.append(sheet_name)
    return visible_sheet_names

# Create a list of visible sheet names from an .xlsx file
def read_visible_sheets_xlsx(file_path):
    visible_sheet_names = []
    workbook = openpyxl.load_workbook(file_path)
    for sheet_name in workbook.sheetnames:
        sheet = workbook[sheet_name]
        if sheet.sheet_state == 'visible':
            visible_sheet_names.append(sheet_name)
    return visible_sheet_names

# Function to check if a date string is in the valid "dd-mm-yy" format
def is_valid_date(date_str):
    try:
        datetime.strptime(date_str, "%d-%m-%y")
        return True
    except ValueError:
        return False

# Function to filter and return valid sheet names based on valid date format
def check_valid_dates(sheet_names):
    invalid_dates = []
    for date in sheet_names:
        if not is_valid_date(date):
            invalid_dates.append(date)
    valid_sheet_names = [item for item in sheet_names if item not in invalid_dates]
    return valid_sheet_names

# Function to check that the valid sheets cover all dates in the given month and identify any missing or extra dates
def check_date_sequence(month, year,valid_sheets):
    year = int(year)
    num_days = monthrange(year, month)[1]
    dates = []
    for day in range(1, num_days + 1):
        dates.append("{:02d}-{:02d}-{}".format(day, month, str(year)[2:]))
        
    set_sheets = set(valid_sheets)
    set_dates = set(dates)
    missing_in_sheets = set(dates) - set(valid_sheets)
    missing_in_dates = set(valid_sheets) - set(dates)
    if dates == valid_sheets:
        check = True
    else:
        if missing_in_sheets:
            check = False
            print("Missing dates:",missing_in_sheets)
        if missing_in_dates:
            check = False
            print("Invalid dates:",missing_in_dates)
    return check

# Function to read .xls file format into a DataFrame, excluding hidden rows and columns
def read_excel_sheet_to_df_xls(sheet, file_path):
    try:
        workbook = xlrd.open_workbook(file_path, formatting_info=True)
        xl_sheet = workbook.sheet_by_name(sheet)
        data = []
        for irow in range(min(xl_sheet.nrows, 50)):
            irow_hidden = xl_sheet.rowinfo_map[irow].hidden   # Row Visibility 0=Visible 1=Hidden
            row_data = []
            for icol in range(xl_sheet.ncols):  # Iterate through all columns in the row
                if icol in xl_sheet.colinfo_map:
                    icol_hidden = xl_sheet.colinfo_map[icol].hidden   # Column Visibility 0=Visible 1=Hidden
                    col_width = xl_sheet.colinfo_map[icol].width
                else:
                    icol_hidden = False  # Column is not explicitly hidden
                    col_width = 2340
                if not irow_hidden and not icol_hidden and col_width > 100:
                    svalue = xl_sheet.cell(irow, icol).value
                    row_data.append(svalue)
            # Only append unhidden rows to the data list
            if not irow_hidden:
                data.append(row_data)
        # Create a DataFrame from the unhidden rows
        df = pd.DataFrame(data)
        return df
    except Exception as e:
        print("An error1 occurred:", e)
        return None

# Function to read .xlsx file format into a DataFrame, excluding hidden rows and columns
def read_excel_sheet_to_df_xlsx(sheet, file_path):
    try:
        # Function to convert Excel column name to numerical value
        def column_to_number(column):
            number = 0
            for char in column:
                number = number * 26 + (ord(char) - 64)
            return number - 1

        # Read Excel file as Pandas DataFrame
        df = pd.read_excel(file_path, sheet_name = sheet )
        
        # Open an Excel workbook
        workbook = openpyxl.load_workbook(file_path)

        # Create a `Worksheet` object 
        worksheet = workbook[sheet]

        # List of indices corresponding to all hidden rows
        hidden_rows_idx = [
            row - 2
            for row, dimension in worksheet.row_dimensions.items() 
            if dimension.hidden
        ]

        # List of indices corresponding to all hidden columns with width less than 1
        hidden_cols_idx = [
            column_to_number(col_name) 
            for col_name, dimension in worksheet.column_dimensions.items() 
            if dimension.hidden or dimension.width < 1
        ]

        # Find names of columns corresponding to hidden column indices
        hidden_cols_name = df.columns[hidden_cols_idx].tolist()

        # Drop the hidden columns
        df.drop(hidden_cols_name, axis=1, inplace=True)

        # Drop the hidden rows
        df.drop(hidden_rows_idx, axis=0, inplace=True)

        # Reset the index
        df.reset_index(drop=True, inplace=True)
        return df
    except Exception as e:
        print("An error1 occurred", e)
        return None

### DataFrame Cleaning, Formatting, and Relevant Data Extraction Through Validation Checks Functions ###

# Extracts and cleans the initial plant readings data from the sheet.
def clean_dataframe1(df):
    try:
        time_index = df[df.iloc[:,0] == "TIME"].index
        if len(time_index)>0:
            start_index = time_index[0]
            end_index = start_index + 26

            df = df.iloc[start_index:end_index]

            df.reset_index(drop=True, inplace=True)

            expected_numbers = list(range(1,25))
            first_column_values = df.iloc[2:, 0].tolist()

            if not first_column_values == expected_numbers:
                incorrect_index = next((i+2 for i, value in enumerate(first_column_values) if value != i+1), None)

                if incorrect_index is not None:
                    for i in range(incorrect_index, len(df)):
                        if pd.notna(df.iloc[i-1,0]) and isinstance(df.iloc[i-1,0], (int,float)):
                            df = df.copy()
                            df.iloc[i, 0] = df.iloc[i-1, 0] + 1
                            
        # Step 1: Ensure consecutive numbers from 1 to 24 in the first column
        expected_numbers = list(range(1, 25))
        if df.iloc[2:, 0].tolist() == expected_numbers:
            # Replace empty strings with NaN
            df = df.replace(r'^\s*$', np.nan, regex=True).copy()  
            
            # Drop columns containing "Calculated" where the rest of the items in that column are NaN
            cols_to_drop = df.columns[df.apply(lambda col: col.astype(str).str.contains('Calculated').any())]
            filtered_cols_to_drop = []
            for column_name in cols_to_drop:
                if df.loc[2:,column_name].apply(lambda x: isinstance(x, str) or pd.isna(x)).all():
                    filtered_cols_to_drop.append(column_name)               
            df = df.drop(columns=filtered_cols_to_drop)          
            # Drop columns with only NaN values
            df = df.dropna(axis=1, how='all')                              
        else:
            print(f"Sheet {sheet} in file {file_name} has an error in their hour order")
            df = None
        return df
    except Exception as e:
        print("An error2 occurred:", e)
        return None


def find_column_index(df):
    try:
        if df is not None:
            # Initialize variables to track the occurrences of "MW" and "WINDING"
            first_MW_index = None
            second_MW_index = None
            third_MW_index = None
            fourth_MW_index = None
            fourth_WINDING_index = None

            # Counter for tracking the number of occurrences found
            occurrence_count1 = 0
            occurrence_count2 = 0
            # Iterate over the columns of the DataFrame
            for column in df.columns:
                # Iterate over the values in each column
                for index, value in enumerate(df[column]):
                    if value == "MW":
                        if first_MW_index is None:
                            first_MW_index = df.columns.get_loc(column)
                            occurrence_count1 += 1
                        elif second_MW_index is None:
                            second_MW_index = df.columns.get_loc(column)
                            occurrence_count1 += 1
                        elif third_MW_index is None:
                            third_MW_index = df.columns.get_loc(column)
                            occurrence_count1 += 1
                        elif fourth_MW_index is None:
                            fourth_MW_index = df.columns.get_loc(column)
                            occurrence_count1 += 1
                        if occurrence_count1 == 4:  # If all four occurrences are found, exit the loop
                            break
                    elif value == "WINDING":
                        occurrence_count2 += 1
                        # Update WINDING index if it's the fourth occurrence
                        if occurrence_count2 == 4:
                            fourth_WINDING_index = df.columns.get_loc(column) + 1
                            break
                if occurrence_count1 == 4 and fourth_WINDING_index is not None:
                    break

            # Store the values in a dictionary
            result = {
                "first_MW_index": first_MW_index,
                "second_MW_index": second_MW_index,
                "third_MW_index": third_MW_index,
                "fourth_MW_index": fourth_MW_index,
                "fourth_WINDING_index": fourth_WINDING_index
            }
        else:
            result = None
        return result
    except Exception as e:
        print("An error3 occurred:", e)
        return None
        
#This function determines the indices of key columns ("MW" and "WINDING") in the DataFrame, which helps in dividing the plant readings into two sections and verifying the order of column headings

def section_check(section_index,df,section,sheet):
    try:
        if section_index is not None:
            if section == 1:
                # Define the pattern to search for
                pattern = re.compile(r'MW|MX|KV|AMP|FIELD', re.IGNORECASE)
                # Labels to check against
                labels = ['MW', 'MX', 'KV', 'AMP', 'FIELD']
                # Variable to track if labels match
                labels_match = True
                a = 5
                b = 0
            elif section == 2:
                # Define the pattern to search for
                pattern = re.compile(r'POWER|RPM|GATE|AVR|OIL|WINDING', re.IGNORECASE)
                # Labels to check against
                labels = ['POWER', 'RPM', 'GATE', 'AVR', 'OIL','WINDING']
                # Variable to track if labels match
                labels_match = True
                a = 8
                b = - 6
            sheet_dict = {sheet: []}
            for key, column in section_index.items():
                # Variable to track the last index of each label seen
                last_index = {label: None for label in labels}
                
                iterations = a  # Number of iterations to the left
                count = 0
                # Loop through columns from start_column_index to the left for the specified number of iterations
                current_index = column + b
                while count < iterations:
                    # Check if the column index exists within df.columns
                    if df.columns[current_index] in df.columns:
                        for i in range(2):  # Iterate through the first two rows
                            cell_value = str(df.iloc[i,current_index])
                            match = pattern.search(cell_value)  # Search for pattern match
                            if match:
                                matched_label = match.group().upper()  # Get the matched label and convert to uppercase
                                if last_index[matched_label] is None:
                                    last_index[matched_label] = current_index
                        current_index +=1
                        count +=1
                    else:
                        # Handle the case where the column index does not exist in df.columns
                        print("Column index", current_index, "does not exist in df.columns.")
                        break
                        
                first_key, current_index = next(iter(last_index.items()))
                if None not in last_index.values() and current_index < 50:
                    for index, (key, value) in enumerate(last_index.items()):
                        if index > 0:  # Skip the first element
                            if value == current_index + 1:
                                current_index = value
                            else:
                                labels_match = False
                                break
                        if not labels_match:
                            break  # Break the outer loop if mismatch found
                    if not labels_match:
                        break  # Break the outer loop if mismatch
                else:
                    key_with_none_value = next(key for key, value in last_index.items() if value is None)
                    print(f"The key '{key_with_none_value}' has a value of None.")
                    labels_match = False
                if labels_match:
                    sheet_dict[sheet].append(list(last_index.values()))

            if not labels_match:
                sheet_dict = None
                print("Correct heading arrangement")

        return sheet_dict
    except Exception as e:
        print(f"{sheet}: An error4 occurred:{e}")
        return None

# Extracts and formats date from the given sheet name
def format_date(sheet,year):
    try:
        parts = sheet.split('-')
        
        # Convert parts to integers
        day = int(parts[0])
        month = int(parts[1])
        year = year[2:]
        
        # Convert back to strings and format them
        formatted_date = "{:02d}/{:02d}/{}".format(month, day, year)
        return formatted_date
    except Exception as e:
        print("An error5 occurred:", e)
        return None

# Generates and appends hourly timestamps to the 'Date&Time' column of the FInal DataFrame based on the provided formatted date
def append_date_time(formatted_date, all_sheets_df):
    try:
        num_rows = len(all_sheets_df)
        # Split the string by '-'
        
        for hour in range(1, 25):
            index = hour - 1
            if hour == 24:
                date_format = "%m/%d/%y"
                date = datetime.strptime(formatted_date, date_format)
                next_day = date + timedelta(days=1)
                next_day_str = next_day.strftime(date_format)
                timestamp = f"{next_day_str} 00:00"
            else:
                timestamp = f"{formatted_date} {hour:02}:00"
            all_sheets_df.at[index + num_rows, 'Date&Time'] = timestamp
        return all_sheets_df
    except Exception as e:
        print("An error6 occurred:", e)
        return None

# This function identifies the column indices that contain voltage readings for the four units
def get_volt_index(combined_index,df,sheet):
    try:
        volt_indices = [ ]
        combined_values = combined_index[sheet]
        indices = [4,5,15,16,26,27,37,38]
        volts_index = [combined_values[index] for index in indices]
        
        # Iterate through pairs of indices
        count = 0
        volt_dict={}
        for i in range(0, len(volts_index), 2):
            start_index = volts_index[i]
            end_index = volts_index[i + 1]

            # Find numbers between the pairs
            volt_dict[i] = [ ]
            for num in range(start_index + 1, end_index):
                volt_dict[i].append(num)
                count +=1
            if start_index + 1 == end_index:
                count += 1
        volt_count = count
        for key,value in volt_dict.items():
            if not volt_dict[key]:
                volt_dict[key].append(np.nan)
                #print(sheet)    
            if volt_count <= 4:
                volt_indices.append(value[0])
            elif volt_count > 4:
                if len(value) > 1:
                    vcount = 0
                    vidx = [ ]
                    for col_idx in value:
                        check = df.iloc[2:, col_idx]
                        has_number = False
                        for val in check:
                            if isinstance(val, (int, float)) and not math.isnan(val):
                                has_number = True
                                break
                        if has_number:
                            vcount += 1
                            vidx.append(col_idx)
                    
                    if vcount == 1:
                        volt_indices.append(vidx[0])
                        
                    elif vcount == 0:
                        volt_indices.append(value[0])
                else:
                    volt_indices.append(value[0])            
        return volt_indices
    except Exception as e:
        print(f"An error7 occurred in {sheet}:", e)
        return None

#This function generates the final processed DataFrame by combining data from the voltage sections and the combined data sections
def append_data(combined_index, df, all_sheets_df,formatted_date,volt_indices):
    try:
        for idx, date_time in all_sheets_df['Date&Time'].items():
            if formatted_date + " 01:00" in date_time:
                start_index = idx
                break
        index1 = 0
        index2 = 0
        for column_name in all_sheets_df.columns[1:]:
            if "VOLTS" not in column_name.upper():
                column_index1 = combined_index[sheet][index1]
                data_to_copy = df.iloc[2:, column_index1]
                all_sheets_df.loc[start_index:start_index + len(data_to_copy), column_name] = data_to_copy.values
                index1 += 1
            elif "VOLTS" in column_name.upper() and len(volt_indices) == 4:
                column_index2 = volt_indices[index2]
                if np.isnan(column_index2):
                    data_to_copy = pd.Series([np.nan] * 24)
                else:
                    data_to_copy = df.iloc[2:, column_index2]
                all_sheets_df.loc[start_index:start_index + len(data_to_copy), column_name] = data_to_copy.values
                index2 += 1
        return all_sheets_df
    except Exception as e:
        print("An error8 occurred:", e)
        return None

### Reading Excel Files to Start the Data Extraction Process ###
months_order = {
    'JAN': 1, 'FEB': 2, 'MAR': 3, 'APR': 4, 'MAY': 5, 'JUN': 6,
    'JUL': 7, 'AUG': 8, 'SEP': 9, 'OCT': 10, 'NOV': 11, 'DEC': 12
}

# Prompt the user to enter the folder path
folder_path = input("Please enter the folder path containing the Excel files: ")
sorted_file_names = sorted(file_names, key=lambda x: (months_order[x[0:3]]))
error_detected = True

### Process and Extract Data from Excel Files in the Specified Folder ###
# Loop through files in the folder
for file_name in sorted_file_names:
    year_index = file_name.find('20')
    year = file_name[year_index:year_index+4]
    month = months_order[file_name[0:3]]
    # Check if the file is an Excel file
    if file_name.endswith('.xlsx') or file_name.endswith('.xls'):      
        file_type = 'xlsx' if file_name.endswith('.xlsx') else 'xls'
        
        # Construct the full path to the file
        file_path = os.path.join(folder_path, file_name)
  
        if file_type == 'xls':

            # Read visible sheets
            sheet_names = read_visible_sheets_xls(file_path)
            
        elif file_type == 'xlsx':
        
            # Read visible sheets
            sheet_names = read_visible_sheets_xlsx(file_path)

        # Check valid dates
        valid_sheets = check_valid_dates(sheet_names)        
        
        check_date = check_date_sequence(month, year,valid_sheets)
        if check_date == False:
            error_detected = False
        
        for sheet in valid_sheets:
            if file_type == 'xls':
                df = read_excel_sheet_to_df_xls(sheet, file_path)
            
            elif file_type == 'xlsx':
                df = read_excel_sheet_to_df_xlsx(sheet, file_path)
                
            df = clean_dataframe1(df)
            
            formatted_date = format_date(sheet,year)
            
            all_sheets_df = append_date_time(formatted_date, all_sheets_df)
            
            if df is not None:
                
                column_index = find_column_index(df)
            
            else:
                column_index = None
            
            if column_index is not None:

                first_section_index = dict(list(column_index.items())[:4])

                first_section = section_check(first_section_index,df,1,sheet)

                second_section_index = dict(list(column_index.items())[1:5])

                second_section = section_check(second_section_index,df,2,sheet)
            
            else:
                first_section = None
                second_section =  None
                
            if first_section is not None and second_section is not None:

                # Initialize combined_index dictionary
                combined_index = {}

                # Iterate over the keys of first_section_index and second_section_index (assuming they have the same keys)
                for key in first_section.keys():
                    # Combine the lists of column indices while maintaining the order
                    combined_index[key] = sum(first_section[key] + second_section[key], [])
                    # Sorting the combined_index
                    combined_index[key].sort()  
                volt_indices = get_volt_index(combined_index,df,sheet)
                
                if len(volt_indices) == 4: 
                    final_sheets_df = append_data(combined_index, df, all_sheets_df,formatted_date,volt_indices)
                    
                else:
                    error_detected = False
                    print(f"Correct Volt Column in {sheet} {file_name}\n")
                
                if final_sheets_df is not None:
                    all_sheets_df = final_sheets_df
                else:
                    error_detected = False
                    print(f"Error in {sheet} {file_name}\n")
            else:
                error_detected = False
                print(f"Error in {sheet} {file_name}\n")

### Final DataFrame Validation and Export to Excel ###
if error_detected: 
    print("No error was detected files successfully compiled")
    
    for column_name in all_sheets_df.columns[1:]:
        for i in range(len(all_sheets_df)):
            if isinstance(all_sheets_df[column_name].iloc[i], str) and not any(char.isdigit() for char in all_sheets_df[column_name].iloc[i]):
                all_sheets_df.at[i, column_name] = np.nan

    all_sheets_df.to_excel(f'{year} Compiled.xlsx', index=False)



No error was detected files successfully compiled
